In [1]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [2]:
AS= pd.read_csv('../data/Endeavor_Air.csv',skiprows=7)


In [3]:
df= pd.DataFrame(AS)
print(len(df))

38476


In [4]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
149,9E,01/03/2018,3349.0,N300PQ,SAV,15:03,00:00
154,9E,01/03/2018,3397.0,N934XJ,CHS,14:00,00:00
157,9E,01/03/2018,3421.0,N819AY,IAD,20:54,00:00
177,9E,01/03/2018,3712.0,N915XJ,PIT,17:44,00:00
201,9E,01/03/2018,4082.0,N903XJ,ORD,21:38,00:00
...,...,...,...,...,...,...,...
37517,9E,12/09/2018,5379.0,N272PQ,RDU,17:55,00:00
37520,9E,12/09/2018,5393.0,N932XJ,RDU,20:04,00:00
37531,9E,12/10/2018,3317.0,N905XJ,RDU,15:50,00:00
37571,9E,12/10/2018,5379.0,N299PQ,RDU,17:55,00:00


In [5]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,9E,01/01/2018,3291.0,N310PQ,SRQ,15:17,14:59
1,9E,01/01/2018,3302.0,N295PQ,DCA,10:23,10:59
2,9E,01/01/2018,3353.0,N840AY,PHL,18:31,18:26
3,9E,01/01/2018,3354.0,N819AY,BWI,19:58,19:37
4,9E,01/01/2018,3390.0,N306PQ,BNA,17:14,16:50


In [6]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [7]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,9E,01/01/2018,3291.0,N310PQ,SRQ,15:17,14:59
1,9E,01/01/2018,3302.0,N295PQ,DCA,10:23,10:59
2,9E,01/01/2018,3353.0,N840AY,PHL,18:31,18:26
3,9E,01/01/2018,3354.0,N819AY,BWI,19:58,19:37
4,9E,01/01/2018,3390.0,N306PQ,BNA,17:14,16:50


In [8]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,9E,2018-01-01,3291.0,N310PQ,SRQ,15:17,14:59
1,9E,2018-01-01,3302.0,N295PQ,DCA,10:23,10:59
2,9E,2018-01-01,3353.0,N840AY,PHL,18:31,18:26
3,9E,2018-01-01,3354.0,N819AY,BWI,19:58,19:37
4,9E,2018-01-01,3390.0,N306PQ,BNA,17:14,16:50
...,...,...,...,...,...,...,...
38470,9E,2018-12-31,5289.0,N905XJ,CVG,17:30,17:10
38471,9E,2018-12-31,5323.0,N840AY,BWI,13:08,12:52
38472,9E,2018-12-31,5351.0,N303PQ,DTW,16:01,15:51
38473,9E,2018-12-31,5358.0,N319PQ,RDU,12:25,12:07


In [9]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,9E,2018-01-01,3291.0,N310PQ,SRQ,15:17,14:59,2018-01-01
1,9E,2018-01-01,3302.0,N295PQ,DCA,10:23,10:59,2018-01-01
2,9E,2018-01-01,3353.0,N840AY,PHL,18:31,18:26,2018-01-01
3,9E,2018-01-01,3354.0,N819AY,BWI,19:58,19:37,2018-01-01
4,9E,2018-01-01,3390.0,N306PQ,BNA,17:14,16:50,2018-01-01
...,...,...,...,...,...,...,...,...
38470,9E,2018-12-31,5289.0,N905XJ,CVG,17:30,17:10,2018-12-31
38471,9E,2018-12-31,5323.0,N840AY,BWI,13:08,12:52,2018-12-31
38472,9E,2018-12-31,5351.0,N303PQ,DTW,16:01,15:51,2018-12-31
38473,9E,2018-12-31,5358.0,N319PQ,RDU,12:25,12:07,2018-12-31


In [10]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
37157,9E,2018-12-02,5406.0,N936XJ,JAX,21:00,24:00,2018-12-02


In [11]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [12]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
149,9E,2018-01-03,3349.0,N300PQ,SAV,15:03,00:00,2018-01-03
154,9E,2018-01-03,3397.0,N934XJ,CHS,14:00,00:00,2018-01-03
157,9E,2018-01-03,3421.0,N819AY,IAD,20:54,00:00,2018-01-03
177,9E,2018-01-03,3712.0,N915XJ,PIT,17:44,00:00,2018-01-03
201,9E,2018-01-03,4082.0,N903XJ,ORD,21:38,00:00,2018-01-03
...,...,...,...,...,...,...,...,...
37517,9E,2018-12-09,5379.0,N272PQ,RDU,17:55,00:00,2018-12-09
37520,9E,2018-12-09,5393.0,N932XJ,RDU,20:04,00:00,2018-12-09
37531,9E,2018-12-10,3317.0,N905XJ,RDU,15:50,00:00,2018-12-10
37571,9E,2018-12-10,5379.0,N299PQ,RDU,17:55,00:00,2018-12-10


In [13]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [14]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [15]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [16]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [17]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,9E,2018-01-01,3291.0,N310PQ,SRQ,15:17,14:59,2018-01-01,2018-01-01 14:59,2018-01-01 15:17
1,9E,2018-01-01,3302.0,N295PQ,DCA,10:23,10:59,2018-01-01,2018-01-01 10:59,2018-01-01 10:23
2,9E,2018-01-01,3353.0,N840AY,PHL,18:31,18:26,2018-01-01,2018-01-01 18:26,2018-01-01 18:31
3,9E,2018-01-01,3354.0,N819AY,BWI,19:58,19:37,2018-01-01,2018-01-01 19:37,2018-01-01 19:58
4,9E,2018-01-01,3390.0,N306PQ,BNA,17:14,16:50,2018-01-01,2018-01-01 16:50,2018-01-01 17:14
...,...,...,...,...,...,...,...,...,...,...
38470,9E,2018-12-31,5289.0,N905XJ,CVG,17:30,17:10,2018-12-31,2018-12-31 17:10,2018-12-31 17:30
38471,9E,2018-12-31,5323.0,N840AY,BWI,13:08,12:52,2018-12-31,2018-12-31 12:52,2018-12-31 13:08
38472,9E,2018-12-31,5351.0,N303PQ,DTW,16:01,15:51,2018-12-31,2018-12-31 15:51,2018-12-31 16:01
38473,9E,2018-12-31,5358.0,N319PQ,RDU,12:25,12:07,2018-12-31,2018-12-31 12:07,2018-12-31 12:25


In [18]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,9E,2018-01-01,3291.0,N310PQ,SRQ,15:17,14:59,2018-01-01,2018-01-01 14:59:00,2018-01-01 15:17:00
1,9E,2018-01-01,3302.0,N295PQ,DCA,10:23,10:59,2018-01-01,2018-01-01 10:59:00,2018-01-01 10:23:00
2,9E,2018-01-01,3353.0,N840AY,PHL,18:31,18:26,2018-01-01,2018-01-01 18:26:00,2018-01-01 18:31:00
3,9E,2018-01-01,3354.0,N819AY,BWI,19:58,19:37,2018-01-01,2018-01-01 19:37:00,2018-01-01 19:58:00
4,9E,2018-01-01,3390.0,N306PQ,BNA,17:14,16:50,2018-01-01,2018-01-01 16:50:00,2018-01-01 17:14:00
...,...,...,...,...,...,...,...,...,...,...
38470,9E,2018-12-31,5289.0,N905XJ,CVG,17:30,17:10,2018-12-31,2018-12-31 17:10:00,2018-12-31 17:30:00
38471,9E,2018-12-31,5323.0,N840AY,BWI,13:08,12:52,2018-12-31,2018-12-31 12:52:00,2018-12-31 13:08:00
38472,9E,2018-12-31,5351.0,N303PQ,DTW,16:01,15:51,2018-12-31,2018-12-31 15:51:00,2018-12-31 16:01:00
38473,9E,2018-12-31,5358.0,N319PQ,RDU,12:25,12:07,2018-12-31,2018-12-31 12:07:00,2018-12-31 12:25:00


In [19]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,9E,2018-01-01,3291.0,N310PQ,SRQ,15:17,14:59,2018-01-01,2018-01-01 14:59:00,2018-01-01 15:17:00,18.0
1,9E,2018-01-01,3302.0,N295PQ,DCA,10:23,10:59,2018-01-01,2018-01-01 10:59:00,2018-01-01 10:23:00,-36.0
2,9E,2018-01-01,3353.0,N840AY,PHL,18:31,18:26,2018-01-01,2018-01-01 18:26:00,2018-01-01 18:31:00,5.0
3,9E,2018-01-01,3354.0,N819AY,BWI,19:58,19:37,2018-01-01,2018-01-01 19:37:00,2018-01-01 19:58:00,21.0
4,9E,2018-01-01,3390.0,N306PQ,BNA,17:14,16:50,2018-01-01,2018-01-01 16:50:00,2018-01-01 17:14:00,24.0
...,...,...,...,...,...,...,...,...,...,...,...
38470,9E,2018-12-31,5289.0,N905XJ,CVG,17:30,17:10,2018-12-31,2018-12-31 17:10:00,2018-12-31 17:30:00,20.0
38471,9E,2018-12-31,5323.0,N840AY,BWI,13:08,12:52,2018-12-31,2018-12-31 12:52:00,2018-12-31 13:08:00,16.0
38472,9E,2018-12-31,5351.0,N303PQ,DTW,16:01,15:51,2018-12-31,2018-12-31 15:51:00,2018-12-31 16:01:00,10.0
38473,9E,2018-12-31,5358.0,N319PQ,RDU,12:25,12:07,2018-12-31,2018-12-31 12:07:00,2018-12-31 12:25:00,18.0


In [20]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 14761
Flights that arrived early in 2019: 9647
Flights that arrived more than 20 mins early in 2019: 3258


In [21]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 23714
Flights that arrived early in 2018: 15593
Flights that arrived more than 20 mins early in 2018: 5436


In [22]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 0
Flights that arrived early in 2017: 0
Flights that arrived more than 20 mins early in 2017: 0


In [23]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 0
Flights that arrived early in 2016: 0
Flights that arrived more than 20 mins early in 2016: 0


In [24]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 0
Flights that arrived early in 2015: 0
Flights that arrived more than 20 mins early in 2015: 0


In [25]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/Endeavor_Air.csv',index=False) 